Imports necessários

In [1]:
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd

Função principal: Raspa dados de uvas processadas no site Embrapa de 1970 até o ano atual.

In [ ]:
def scrape_all_processamento():
    print("Iniciando raspagem...\n")
    resultados = []
    current_year = datetime.datetime.now().year
    base_url = "http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_03&ano={ano}"

    for ano in range(1970, current_year + 1):
        print(f"Raspando dados do ano {ano}...")

        try:
            response = requests.get(base_url.format(ano=ano))
            response.raise_for_status()
        except requests.RequestException as e:
            print(f"Erro ao acessar dados de {ano}: {e}")
            continue

        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'class': 'tb_base tb_dados'})
        if not table:
            print(f"Sem dados disponíveis para o ano {ano}. Pulando...")
            continue

        rows = table.find_all('tr')
        if not rows:
            print(f"Nenhuma linha encontrada para o ano {ano}.")
            continue

        categoria = None
        rows = rows[:-1]

        for i, row in enumerate(rows):
            if i == 0:
                continue

            cols = row.find_all(['th', 'td'])
            dados = [col.get_text(strip=True) for col in cols]

            if len(dados) == 2:
                produto, quantidade = dados

                if produto.isupper():
                    categoria = produto
                    print(f"Categoria encontrada: {categoria}")
                    continue

                if quantidade == "-":
                    quantidade = "0"

                tipo = f"{categoria} {produto}" if categoria else produto
                resultados.append({
                    "ano": ano,
                    "tipo": tipo,
                    "quantidade": quantidade
                })
                print(f"Ano: {ano} | Tipo: {tipo} | Quantidade: {quantidade}")

    print(f"Raspagem concluída. Total de registros: {len(resultados)}")
    return resultados


Criação do DataFrame

In [3]:
dados = scrape_all_processamento()
df = pd.DataFrame(dados)
df.head()

Iniciando raspagem...

Raspando dados do ano 1970...
Categoria encontrada: TINTAS
Ano: 1970 | Tipo: TINTAS Alicante Bouschet | Quantidade: 0
Ano: 1970 | Tipo: TINTAS Ancelota | Quantidade: 0
Ano: 1970 | Tipo: TINTAS Aramon | Quantidade: 0
Ano: 1970 | Tipo: TINTAS Alfrocheiro | Quantidade: 0
Ano: 1970 | Tipo: TINTAS Arinarnoa | Quantidade: 0
Ano: 1970 | Tipo: TINTAS Aspirant Bouschet | Quantidade: 0
Ano: 1970 | Tipo: TINTAS Barbera | Quantidade: 4.548.313
Ano: 1970 | Tipo: TINTAS Bonarda | Quantidade: 1.631.610
Ano: 1970 | Tipo: TINTAS Cabernet Franc | Quantidade: 238.837
Ano: 1970 | Tipo: TINTAS Cabernet Sauvignon | Quantidade: 0
Ano: 1970 | Tipo: TINTAS Caladoc | Quantidade: 0
Ano: 1970 | Tipo: TINTAS Campanario | Quantidade: 0
Ano: 1970 | Tipo: TINTAS Canaiolo | Quantidade: 596.766
Ano: 1970 | Tipo: TINTAS Carignan | Quantidade: 0
Ano: 1970 | Tipo: TINTAS Carmenere | Quantidade: 1.235
Ano: 1970 | Tipo: TINTAS Castelão | Quantidade: 0
Ano: 1970 | Tipo: TINTAS Corvina | Quantidade: 0
A

,ano,tipo,quantidade
0,1970,TINTAS Alicante Bouschet,0
1,1970,TINTAS Ancelota,0
2,1970,TINTAS Aramon,0
3,1970,TINTAS Alfrocheiro,0
4,1970,TINTAS Arinarnoa,0


Salvamento do df em csv

In [4]:
df.to_csv("dados_processamento_embrapa.csv", index=False)
print("Arquivo CSV salvo com sucesso.")

Arquivo CSV salvo com sucesso.
